In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
import requests
import pandas as pd

api_key = os.environ.get('riot_api_key')

def get_puuid(summonerId=None, gameName=None, tagLine=None, region='americas'):
    """Gets the puuid from a summonerId or riot_id and riot_tag

    Args:
        summonerId(str, opitional): Summoner ID, Defaults to None.
        gameName(str, opitional): Riot ID, Defaults to None. 
        tagLine(str, opitional): Riot Tag, Defaults to None. 
        region(str, opitional): Region, Defaults to 'americas'.

    returns: 
        str: puuid
    """
    if summonerId is not None: 
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        print(root_url+endpoint+summonerId+'?api_key='+api_key)
        response = requests.get(root_url+endpoint+summonerId+'?api_key='+api_key)

        return response.json()['puuid']
    else:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'

        response = requests.get(root_url+endpoint+'?api_key='+api_key)

        return response.json()['puuid']
    
def get_idtag_from_puuid(puuid=None):
    """Gets the riot_id and riot_tag from puuid

    Args:
        puuid(str, optional): puuid, Defaults to None.
    
    Returns:
        id(dict): Dictionary with riot_id and riot_tag.
    """
    
    root_url = 'https://americas.api.riotgames.com'
    endpoint = 'riot/account/v1/accounts/by-puuid/'

    response = requests.get(root_url+endpoint+puuid+'?api_key='+api_key)

    id = {
        'gameName': response.json()['gameName'],
        'tagLine': response.json()['tagLine']
    }

    return id

def get_ladder(top=300):
    """ Gets the top X players in soloQ.

    Args:
        top(int, optional) Number of players to return. Defaults to 300.
    Returns: 
        DataFrame: Returns a DataFrame of the top X players in soloq.
    """

    root_url = 'https://br1.api.riotgames.com/'
    challenger = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    grandmaster = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    master = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'

    chall_response = requests.get(root_url + challenger + '?api_key=' + api_key)
    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)

    gm_df = pd.DataFrame()
    master_df = pd.DataFrame()
    if top > 300:    
        gm_response = requests.get(root_url + grandmaster + '?api_key=' + api_key)
        gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    if top > 1000:
        masters_response = requests.get(root_url + master + '?api_key=' + api_key)
        master_df = pd.DataFrame(masters_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    
    df = pd.concat([chall_df, gm_df, master_df]).reset_index(drop=True)[:top]

    df = df.reset_index().drop(columns=['rank']).rename(columns={'index': 'rank'})
    df['rank'] += 1

    return df

def get_match_history(region=None, puuid=None, start=0, count=20):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'/lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}'

    response = requests.get(root_url+endpoint+query_params+'&api_key='+api_key)

    return response.json()

def get_match_data_from_id(region=None, match_Id=None):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'/lol/match/v5/matches/{match_Id}'
    print(root_url+endpoint+'?api_key='+api_key)
    response = requests.get(root_url+endpoint+'?api_key='+api_key)

    return response.json()

def process_match_json(match_json, puuid):              
    ## Architecture
    metadata = match_json['metadata']
    info = match_json['info']
    players = info['participants']    
    participants = game['metadata']['participants']
    teams = info['teams']
    player = players[participants.index(puuid)]
    perks = player['perks']
    stats = perks['statPerks']
    styles = perks['styles']

    primary = styles[0]
    secondary = styles[1]

    side_dict = {
        100:'blue',
        200:'red'
    }

    ## Information
    match_Id = match_json['metadata']['matchId']

    game_creation = info['gameCreation']
    game_duration = info['gameDuration']
    game_start_timestamp = info['gameStartTimestamp']
    game_end_timestamp = info['gameEndTimestamp']
    patch = info['gameVersion']

    riot_id = player['riotIdGameName']
    riot_tag =player['riotIdTagline']
    summoner_id = player['summonerId']
    summoner_name = player['summonerName']

    side = side_dict[player['teamId']]
    win = player['win']

    champ_id = player['championId']
    champ_transform = player['championTransform']
    champ_level = player['champLevel']

    gold_earned = player['goldEarned']
    neutral_minion_killed = player['neutralMinionsKilled']
    total_minions_killed = player['totalMinionsKilled']
    total_ally_jungle_minions_killed = player['totalAllyJungleMinionsKilled']
    total_enemy_jungle_minions_killed = player['totalEnemyJungleMinionsKilled']

    kills = player['kills']
    assists = player['assists']
    deaths = player['deaths']
    first_blood = player['firstBloodKill']

    totel_damage_delt = player['totalDamageDealt']
    total_damage_shilded = player['totalDamageShieldedOnTeammates']
    total_damage_taken = player['totalDamageTaken']
    total_damage_healed = player['totalHealsOnTeammates']
    total_time_cc_dealt = player['totalTimeCCDealt']

    early_surrender = player['gameEndedInEarlySurrender']
    surrender = player['gameEndedInSurrender']

    item0 = player['item0']
    item1 = player['item1']
    item2 = player['item2']
    item3 = player['item3']
    item4 = player['item4']
    item5 = player['item5']
    item6 = player['item6']

    summoner_1_id = player['summoner1Id']
    summoner_2_id = player['summoner2Id']

    wards_placed = player['wardsPlaced']
    wards_killed = player['wardsKilled']
    vision_score = player['visionScore']

    #what is this? 
    detector_wards_placed = player['detectorWardsPlaced']
    role = player['role']

    defense = stats['defense']
    flex = stats['flex']
    offense = stats['offense']

    primary_style = primary['style']
    secondary_style = secondary['style']

    primary_keystone = primary['selections'][0]['perk']
    primary_perk_1 = primary['selections'][1]['perk']
    primary_perk_2 = primary['selections'][2]['perk']
    primary_perk_3 = primary['selections'][3]['perk']

    secondary_perk_1 = secondary['selections'][0]['perk']
    secondary_perk_2 = secondary['selections'][1]['perk']

    objective_stolen = player['objectivesStolen']
    objective_stolen_assists = player['objectivesStolenAssists']


    matchDF = pd.DataFrame({
        'match_Id': [match_Id],
        'participants': [participants],
        'game_creation': [game_creation],
        'game_duration': [game_duration],
        'game_start_timestamp': [game_start_timestamp],
        'game_end_timestamp': [game_end_timestamp],
        'patch': [patch],
        'puuid': [puuid],
        'riot_id': [riot_id],
        'riot_tag': [riot_tag],
        'summoner_id': [summoner_id],
        'summoner_name': [summoner_name],
        'win': [win],
        'champion': [champ_id],
        'champ_transform': [champ_transform],
        'champ_level': [champ_level],
        'kills': [kills],
        'deaths': [deaths],
        'assists':[assists],
        'summoner1_id': [summoner_1_id],
        'summoner2_id': [summoner_2_id],
        'gold_earned': [gold_earned],
        'total_minions_killed': [total_minions_killed],
        'total_neutral_minions_killed': [neutral_minion_killed],
        'total_ally_jungle_minions_killed': [total_ally_jungle_minions_killed],
        'total_enemy_jungle_minions_killed': [total_enemy_jungle_minions_killed],
        'early_surrender': [early_surrender],
        'surrender': [surrender],
        'first_blood': [first_blood],
        'objectives_stolen': [objective_stolen],
        'objective_stonlen_assists': [objective_stolen_assists],
        'total_damage_delt_champions': [totel_damage_delt],
        'total_damage_taken': [total_damage_taken],
        'total_damage_shilded_teammates': [total_damage_shilded],
        'total_heals_teammates': [total_damage_healed],
        'total_time_crowd_controlled': [total_time_cc_dealt],
        'vision_score': [vision_score],
        'wards_killed': [wards_killed],
        'wards_placed': [wards_placed],
        'control_wards_placed': [detector_wards_placed],
        'item0': [item0],
        'item1': [item1],
        'item2': [item2],
        'item3': [item3],
        'item4': [item4],
        'item5': [item5],
        'item6': [item6],
        'park_primary_row_1': [primary_perk_1],
        'park_primary_row_2': [primary_perk_2],
        'park_primary_row_3': [primary_perk_3],
        'park_secondary_row_1': [secondary_perk_1],
        'park_secondary_row_2': [secondary_perk_2],
        'perk_primary_style': [primary_style],
        'perk_secondary_style': [secondary_style],
        'park_shard_defense': [defense],
        'park_shard_flex': [flex],
        'park_shard_offense': [offense],
        
    })
    return matchDF

In [2]:
get_ladder(2000)

,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,Oek5G0_0_if_tmyR6FIQ4qu7KELVj2vLeRfeGujPOuOahJ...,1697,165,108,True,False,False,True
1,2,jlrnYK1wjlFUH15V7p6The_r_AZo0NaogvVlFUba8PBW,1661,309,260,True,False,False,False
2,3,mJeIYif9KVsFZ61unGVxqZcetiKwEi6bq0Whx_EQtqdnW5o,1653,132,67,True,False,False,True
3,4,z0px8f_1exj4jMA6rQ-7yAzzDnBkcxOIotfeg4ZpqH2sQA,1587,239,186,True,False,False,False
4,5,0odymtFEIiL6LYiDr7_RMysWmqAfS5RtCxYBi13FDLUTAYI,1565,245,196,True,False,False,True
...,...,...,...,...,...,...,...,...,...
1995,1996,ZU7jWK26OqmzBxZZcYhw7taw5BU12IV8tcCm8njnssGa_s4,28,89,87,False,False,False,False
1996,1997,jVvcZUYqioGJXongIjQxPsZD2aIFisDqXx0mk-Mj9drZ0Q,27,21,11,False,False,True,False
1997,1998,AIuRUqpos57D4N6qpAbHszqd6tf0k7LS0RrRxV9ClGdx,27,27,13,False,False,True,False
1998,1999,TBU9d8AAkHYjJ96SNCS0-5499wyAjxEoYnoKdBGW4tfR3P...,27,65,51,False,False,False,False


In [3]:
get_match_history(region='americas', puuid='GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFeLZSZiXhgYmuWOE_LhexsXRJaFmD7MhEw', start=0, count=100)

['BR1_3025501078',
 'BR1_3025499053',
 'BR1_3025473478',
 'BR1_3025465419',
 'BR1_3025439341',
 'BR1_3025404081',
 'BR1_3025374799',
 'BR1_3025335200',
 'BR1_3025317262',
 'BR1_3025299570',
 'BR1_3025279840',
 'BR1_3025258886',
 'BR1_3025246532',
 'BR1_3025124039',
 'BR1_3025116615',
 'BR1_3025104628',
 'BR1_3025092120',
 'BR1_3025077447',
 'BR1_3025067753',
 'BR1_3025047835',
 'BR1_3025028907',
 'BR1_3025003409',
 'BR1_3024967919',
 'BR1_3024943783',
 'BR1_3024917788',
 'BR1_3024888380',
 'BR1_3024866618',
 'BR1_3024834911',
 'BR1_3024814326',
 'BR1_3024639719',
 'BR1_3024629874',
 'BR1_3024599758',
 'BR1_3024578217',
 'BR1_3024545605',
 'BR1_3024518839',
 'BR1_3024499166',
 'BR1_3024492070',
 'BR1_3024453824',
 'BR1_3024176578',
 'BR1_3024152270',
 'BR1_3024136855',
 'BR1_3024114733',
 'BR1_3024082582',
 'BR1_3024059371',
 'BR1_3024025728',
 'BR1_3024000034',
 'BR1_3023969506',
 'BR1_3023953481',
 'BR1_3023705210',
 'BR1_3023688894',
 'BR1_3023670881',
 'BR1_3023654917',
 'BR1_302362

In [4]:
match_ids = get_match_history(region='americas', puuid='GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFeLZSZiXhgYmuWOE_LhexsXRJaFmD7MhEw')

df = pd.DataFrame()
for match_id in match_ids:
    game = get_match_data_from_id(region='americas', match_Id=match_id)
    matchDF = process_match_json(game, puuid='GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFeLZSZiXhgYmuWOE_LhexsXRJaFmD7MhEw')
    
    df = pd.concat([df, matchDF])

https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3025501078?api_key=RGAPI-bedfe1db-60b5-4b9e-bf4e-d9b367b64cc2
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3025499053?api_key=RGAPI-bedfe1db-60b5-4b9e-bf4e-d9b367b64cc2
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3025473478?api_key=RGAPI-bedfe1db-60b5-4b9e-bf4e-d9b367b64cc2
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3025465419?api_key=RGAPI-bedfe1db-60b5-4b9e-bf4e-d9b367b64cc2
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3025439341?api_key=RGAPI-bedfe1db-60b5-4b9e-bf4e-d9b367b64cc2
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3025404081?api_key=RGAPI-bedfe1db-60b5-4b9e-bf4e-d9b367b64cc2
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3025374799?api_key=RGAPI-bedfe1db-60b5-4b9e-bf4e-d9b367b64cc2
https://americas.api.riotgames.com//lol/match/v5/matches/BR1_3025335200?api_key=RGAPI-bedfe1db-60b5-4b9e-bf4e-d9b367b64cc2
https://americas

In [5]:
df

,match_Id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,...,park_primary_row_1,park_primary_row_2,park_primary_row_3,park_secondary_row_1,park_secondary_row_2,perk_primary_style,perk_secondary_style,park_shard_defense,park_shard_flex,park_shard_offense
0,BR1_3025501078,[N9yldcjdXYcqCtUaQNNP5eSzrCDWkb1pTh_kkuPhyycOx...,1731999224988,1585,1731999305654,1732000891289,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,8143,8138,8135,9111,9104,8100,8000,5001,5008,5005
0,BR1_3025499053,[aiwqWO4n7gzDs3ZTr_axgjLVBqzmi9bbTub0cSIAbgfL1...,1731996537307,917,1731996558231,1731997475155,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9104,8014,8304,8347,8000,8300,5001,5008,5005
0,BR1_3025473478,[LZXsOCkjSHU85NTPvjRlrgGp_7Rm7K7Gjjn6F12K_85XW...,1731987905181,1411,1731987924320,1731989335919,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,8143,8138,8135,9111,9104,8100,8000,5001,5008,5005
0,BR1_3025465419,[avrSbjOnb_nXztDCVt-d8ajA41fKQTQO6Y9m0Ow5AoFOv...,1731986283562,104,1731986365522,1731986470144,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9104,8014,8304,8347,8000,8300,5001,5008,5005
0,BR1_3025439341,[HaZjwnBChtxUY6YLlpJ4eI4Q8lFhD8jTLUwH9zkh0_X-8...,1731982543609,1771,1731982576077,1731984346956,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,8143,8138,8135,9104,9111,8100,8000,5001,5008,5005
0,BR1_3025404081,[yOvDDEJPXj_AXwxJSIf7EEyuiIzzwV8_hNEX60cqJg6cz...,1731978578061,2518,1731978611324,1731981129601,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9104,8014,8304,8347,8000,8300,5001,5008,5005
0,BR1_3025374799,[yOvDDEJPXj_AXwxJSIf7EEyuiIzzwV8_hNEX60cqJg6cz...,1731975960964,1599,1731976112252,1731977711190,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9104,8014,8304,8347,8000,8300,5001,5008,5005
0,BR1_3025335200,[jp5grFdjV7jBI35sHYSvgHzu_uNndNUDrXA_IooS_UfPF...,1731971641134,2221,1731971662798,1731973884227,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9105,8014,8210,8232,8000,8200,5001,5008,5005
0,BR1_3025317262,[jp5grFdjV7jBI35sHYSvgHzu_uNndNUDrXA_IooS_UfPF...,1731969469859,1507,1731969494298,1731971001836,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9105,8014,8210,8232,8000,8200,5001,5008,5005
0,BR1_3025299570,[vZVecgX4WNuJN8VR3mFThbNun6I9IL8-10QHiNnfYn8IA...,1731967453458,1564,1731967470965,1731969035078,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9105,8014,8210,8232,8000,8200,5001,5008,5005


In [8]:
temp_dict = {
    8143: 'Perk 1',
    8401: 'Perk 2',
}

In [9]:
df.replace(temp_dict)

,match_Id,participants,game_creation,game_duration,game_start_timestamp,game_end_timestamp,patch,puuid,riot_id,riot_tag,...,park_primary_row_1,park_primary_row_2,park_primary_row_3,park_secondary_row_1,park_secondary_row_2,perk_primary_style,perk_secondary_style,park_shard_defense,park_shard_flex,park_shard_offense
0,BR1_3025501078,[N9yldcjdXYcqCtUaQNNP5eSzrCDWkb1pTh_kkuPhyycOx...,1731999224988,1585,1731999305654,1732000891289,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,Perk 1,8138,8135,9111,9104,8100,8000,5001,5008,5005
0,BR1_3025499053,[aiwqWO4n7gzDs3ZTr_axgjLVBqzmi9bbTub0cSIAbgfL1...,1731996537307,917,1731996558231,1731997475155,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9104,8014,8304,8347,8000,8300,5001,5008,5005
0,BR1_3025473478,[LZXsOCkjSHU85NTPvjRlrgGp_7Rm7K7Gjjn6F12K_85XW...,1731987905181,1411,1731987924320,1731989335919,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,Perk 1,8138,8135,9111,9104,8100,8000,5001,5008,5005
0,BR1_3025465419,[avrSbjOnb_nXztDCVt-d8ajA41fKQTQO6Y9m0Ow5AoFOv...,1731986283562,104,1731986365522,1731986470144,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9104,8014,8304,8347,8000,8300,5001,5008,5005
0,BR1_3025439341,[HaZjwnBChtxUY6YLlpJ4eI4Q8lFhD8jTLUwH9zkh0_X-8...,1731982543609,1771,1731982576077,1731984346956,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,Perk 1,8138,8135,9104,9111,8100,8000,5001,5008,5005
0,BR1_3025404081,[yOvDDEJPXj_AXwxJSIf7EEyuiIzzwV8_hNEX60cqJg6cz...,1731978578061,2518,1731978611324,1731981129601,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9104,8014,8304,8347,8000,8300,5001,5008,5005
0,BR1_3025374799,[yOvDDEJPXj_AXwxJSIf7EEyuiIzzwV8_hNEX60cqJg6cz...,1731975960964,1599,1731976112252,1731977711190,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9104,8014,8304,8347,8000,8300,5001,5008,5005
0,BR1_3025335200,[jp5grFdjV7jBI35sHYSvgHzu_uNndNUDrXA_IooS_UfPF...,1731971641134,2221,1731971662798,1731973884227,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9105,8014,8210,8232,8000,8200,5001,5008,5005
0,BR1_3025317262,[jp5grFdjV7jBI35sHYSvgHzu_uNndNUDrXA_IooS_UfPF...,1731969469859,1507,1731969494298,1731971001836,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9105,8014,8210,8232,8000,8200,5001,5008,5005
0,BR1_3025299570,[vZVecgX4WNuJN8VR3mFThbNun6I9IL8-10QHiNnfYn8IA...,1731967453458,1564,1731967470965,1731969035078,14.22.635.4945,GKKegb3YqLoWBIQMwQvErzMjMgEGQz9hp7UQP4A8gfdpFe...,paiN Tatu,10y,...,9111,9105,8014,8210,8232,8000,8200,5001,5008,5005


In [11]:
perk = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perks.json'
perk_styles = 'https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/perkstyles.json'

perk_json = requests.get(perk).json()
perk_styles_json = requests.get(perk_styles).json()

In [12]:
perk_json

[{'id': 8369,
  'name': 'First Strike',
  'majorChangePatchVersion': '11.23',
  'tooltip': 'Attacks or abilities against an enemy champion within @GraceWindow.2@s of entering champion combat grant @GoldProcBonus@ gold and <b>First Strike</b> for @Duration@ seconds, causing you to deal <truedamage>@DamageAmp*100@%</truedamage> extra <truedamage>damage</truedamage> against champions, and granting <gold>{{ Item_Melee_Ranged_Split }}</gold> of that damage dealt as <gold>gold</gold>.<br><br>Cooldown: <scaleLevel>@Cooldown@</scaleLevel>s<br><hr><br>Damage Dealt: @f1@<br>Gold Gained: @f2@',
  'shortDesc': 'When you initiate champion combat, deal 7% extra damage for 3 seconds and gain gold based on damage dealt.',
  'longDesc': 'Attacks or abilities against an enemy champion within 0.25s of entering champion combat grant 10 gold and <b>First Strike</b> for 3 seconds, causing you to deal <truedamage>7%</truedamage> extra <truedamage> damage</truedamage> against champions, and granting <gold>50%

In [15]:
def json_extract(obj, key):

    arr = []

    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict, list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)

        return arr
    
    values = extract(obj, arr, key)
    return values

In [ ]:
perk_ids = json_extract(perk_json, 'id')
perk_names = json_extract(perk_json, 'name')

perk_dict = dict(map(lambda i, j: (i, j), perk_ids, perk_names))

In [20]:
perk_dict

{8369: 'First Strike',
 8446: 'Demolish',
 8126: 'Cheap Shot',
 8415: 'The Arcane Colossus',
 8410: 'Approach Velocity',
 8232: 'Waterwalking',
 8299: 'Last Stand',
 8112: 'Electrocute',
 8234: 'Celerity',
 8453: 'Revitalize',
 8360: 'Unsealed Spellbook',
 8004: 'The Brazen Perfect',
 8128: 'Dark Harvest',
 8220: 'The Calamity',
 8016: 'The Merciless Elite',
 8473: 'Bone Plating',
 8339: 'Celestial Body',
 8214: 'Summon Aery',
 8237: 'Scorch',
 8139: 'Taste of Blood',
 9101: 'Absorb Life',
 8008: 'Lethal Tempo',
 8010: 'Conqueror',
 9105: 'Legend: Haste',
 8106: 'Ultimate Hunter',
 8017: 'Cut Down',
 8224: 'Nullifying Orb',
 8210: 'Transcendence',
 8005: 'Press the Attack',
 8435: 'Mirror Shell',
 8115: 'The Aether Blade',
 8359: 'Kleptomancy',
 8352: 'Time Warp Tonic',
 5003: 'Magic Resist',
 8135: 'Treasure Hunter',
 8120: 'Ghost Poro',
 8134: 'Ingenious Hunter',
 8351: 'Glacial Augment',
 8242: 'Unflinching',
 8401: 'Shield Bash',
 9111: 'Triumph',
 8105: 'Relentless Hunter',
 8454: